In [ ]:
import sys
import logging

In [7]:
# Install dependencies
# !pip install InstructorEmbedding torch transformers sentence-transformers
# !pip uninstall -y transformers sentence-transformers InstructorEmbedding
# !pip install transformers==4.31.0 sentence-transformers==2.2.2 InstructorEmbedding
# !pip install wikipedia
# %pip install ipython-ngql nebula3-python
# %pip install llama-index-graph-stores-neo4j
# %pip install neo4j



In [ ]:
import requests
from typing import Any, List
from llama_index.core.embeddings import BaseEmbedding
from llama_index.core.bridge.pydantic import PrivateAttr
from llama_index.core import KnowledgeGraphIndex
from llama_index.graph_stores.nebula import NebulaGraphStore
from IPython.display import Markdown, display
from llama_index.core import Settings

In [11]:
# This is a custom embedding class for remote instructor embeddings
# It uses the requests library to make HTTP POST requests to a remote server
# to get the embeddings for the provided texts.
# The class is initialized with the base URL of the server, an API key for authentication,
# the model name, and an instruction string.


class RemoteInstructorEmbeddings(BaseEmbedding):
    _base_url: str = PrivateAttr()
    _api_key: str = PrivateAttr()
    _model_name: str = PrivateAttr()
    _instruction: str = PrivateAttr()

    def __init__(
        self,
        base_url: str,
        api_key: str,
        model_name: str = "snowflake-arctic-embed-s",  # Change as needed
        # model_name: str, 
        instruction: str = "Represent the text for semantic search:",
        **kwargs: Any,
    ):
        super().__init__(**kwargs)
        self._base_url = base_url.rstrip("/")
        self._api_key = api_key
        self._model_name = model_name
        self._instruction = instruction

    def _post_request(self, texts: List[str]) -> List[List[float]]:
        headers = {
            "Authorization": f"Bearer {self._api_key}",
            "Content-Type": "application/json"
        }

        # If you want to keep using instructions
        inputs = [f"{self._instruction} {text}" for text in texts]

        payload = {
            "model": self._model_name,
            "input": inputs if len(inputs) > 1 else inputs[0]  # API accepts string or list
        }

        response = requests.post(
            f"{self._base_url}/embeddings",
            headers=headers,
            json=payload
        )
        response.raise_for_status()
        return response.json()["data"]
    
    async def _aget_query_embedding(self, query: str) -> List[float]:
        return self._get_query_embedding(query)

    async def _aget_text_embedding(self, text: str) -> List[float]:
        return self._get_text_embedding(text)
    
    def _get_query_embedding(self, query: str) -> List[float]:
        return self._post_request([query])[0]

    def _get_text_embedding(self, text: str) -> List[float]:
        return self._post_request([text])[0]

    def _get_text_embeddings(self, texts: List[str]) -> List[List[float]]:
        return self._post_request(texts)

In [ ]:
# Initialize the embedding models with the remote server details

embed_model_snow = RemoteInstructorEmbeddings(
    base_url="http://llms-inference.innkube.fim.uni-passau.de",
    api_key="enter the api key",  # Replace with your actual API key
    model_name="Snowflake/snowflake-arctic-embed-xs",  # or whatever model your server uses
)

embed_model_intfloat = RemoteInstructorEmbeddings(
    base_url="http://llms-inference.innkube.fim.uni-passau.de",
    api_key="enter the api key",
    model_name="intfloat/multilingual-e5-large",  # or whatever model your server uses
)

embed_model_jinaai = RemoteInstructorEmbeddings(
    base_url="http://llms-inference.innkube.fim.uni-passau.de",
    api_key="enter the api key",
    model_name="jinaai/jina-embeddings-v2-base-de",  # or whatever model your server uses
)


# Example usage
sentence = "What did the author do growing up?"



In [ ]:
# Chekking if the embedding model is working
# This will return the embedding vector for the given sentence
embedding_snow = embed_model_snow._get_query_embedding(sentence)

print(f"Embedding vector:\n{embedding_snow}")

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): llms-inference.innkube.fim.uni-passau.de:80
DEBUG:urllib3.connectionpool:http://llms-inference.innkube.fim.uni-passau.de:80 "POST /embeddings HTTP/1.1" 308 164
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): llms-inference.innkube.fim.uni-passau.de:443
DEBUG:urllib3.connectionpool:https://llms-inference.innkube.fim.uni-passau.de:443 "POST /embeddings HTTP/1.1" 200 4883
Embedding vector:
{'object': 'embedding', 'index': 0, 'embedding': [0.03755944, 0.03179579, -0.0016535937, -0.006316622, -0.007491683, -0.0060295025, -0.088815525, -0.019279521, 0.062740855, 0.013409528, -0.0010979649, 0.09170799, -0.0106818965, -0.044152547, 0.015897894, 0.018162945, -0.045811456, 0.084051475, -0.10999854, -0.017663145, -0.014483566, 0.08315822, -0.0026611693, 0.029541371, -0.016801788, 0.02788246, -0.047512904, 0.0020483825, 0.013643477, -0.094855666, 0.043025337, 0.011601741, -0.012261052, -0.037495635, -0.027818657, -0.045

In [ ]:
# Chekking if the embedding model is working
# This will return the embedding vector for the given sentence
embedding_intfloat = embed_model_intfloat._get_query_embedding(sentence)

print(f"Embedding vector:\n{embedding_intfloat}")

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): llms-inference.innkube.fim.uni-passau.de:80
DEBUG:urllib3.connectionpool:http://llms-inference.innkube.fim.uni-passau.de:80 "POST /embeddings HTTP/1.1" 308 164
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): llms-inference.innkube.fim.uni-passau.de:443
DEBUG:urllib3.connectionpool:https://llms-inference.innkube.fim.uni-passau.de:443 "POST /embeddings HTTP/1.1" 200 12859
Embedding vector:
{'object': 'embedding', 'index': 0, 'embedding': [-0.0053951642, -0.013991855, -0.02413849, -0.046007115, 0.026205083, -0.035877418, -0.00933355, 0.08340907, 0.02869516, -0.03106666, 0.026018752, 0.025730783, -0.049191702, -0.034064915, -0.0435001, -0.01627866, -0.02437564, 0.046040993, 0.005115666, -0.011518719, 0.026679384, -0.023511736, -0.02866128, -0.0329808, -0.007199199, -0.003658887, -0.027763499, -0.012416501, -0.012585893, -0.04949661, -0.016837656, -0.009934895, -0.037097048, -0.011459431, -0.047430016, 0.03318407

In [ ]:
# Chekking if the embedding model is working
# This will return the embedding vector for the given sentence
embedding_jinaai = embed_model_jinaai._get_query_embedding(sentence)

print(f"Embedding vector:\n{embedding_jinaai}")

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): llms-inference.innkube.fim.uni-passau.de:80
DEBUG:urllib3.connectionpool:http://llms-inference.innkube.fim.uni-passau.de:80 "POST /embeddings HTTP/1.1" 308 164
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): llms-inference.innkube.fim.uni-passau.de:443
DEBUG:urllib3.connectionpool:https://llms-inference.innkube.fim.uni-passau.de:443 "POST /embeddings HTTP/1.1" 200 9678
Embedding vector:
{'object': 'embedding', 'index': 0, 'embedding': [0.01325042, -0.074396856, -0.024057414, 0.046972133, 0.032117072, -0.039483823, -0.06496353, 0.036031418, 0.004974986, 0.02041051, -0.03136338, -0.023328034, 0.041890778, -0.050327286, 0.04631569, 0.0415504, 0.014222928, -0.004838227, 0.03702824, -0.010460538, -0.0030314894, -0.033624463, -0.03722274, 0.017371424, 0.014089208, 0.0014466055, -0.012715541, 0.011159528, -0.014830746, -0.033429958, 0.032433137, -0.08485132, 0.095500275, -0.044808302, 0.017492985, -0.0070871515, -0

In [16]:
import os
from llama_index.core import download_loader

In [ ]:
# Here we are using the WikipediaReader to fetch data from Wikipedia
# Make sure you have the necessary packages installed:  

# Load the WikipediaReader
WikipediaReader = download_loader("WikipediaReader")
loader = WikipediaReader()

# Define the page you want to fetch
page_title = 'Guardians of the Galaxy Vol. 3'

# Load data from Wikipedia
documents = loader.load_data(pages=[page_title], auto_suggest=False)

# Prepare folder and filename
folder_name = 'wiki_data'
os.makedirs(folder_name, exist_ok=True)

# Clean filename (remove special characters, keep it safe)
safe_filename = page_title.replace(" ", "_").replace("/", "_") + ".txt"
file_path = os.path.join(folder_name, safe_filename)

# Save the text content
with open(file_path, "w", encoding="utf-8") as f:
    for doc in documents:
        f.write(doc.text + "\n\n")

print(f"Saved to: {file_path}")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_10280\332848010.py:2: DeprecationWarning: Call to deprecated function (or staticmethod) download_loader. (`download_loader()` is deprecated. Please install tool using pip install directly instead.)
  WikipediaReader = download_loader("WikipediaReader")


DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): en.wikipedia.org:80
DEBUG:urllib3.connectionpool:http://en.wikipedia.org:80 "GET /w/api.php?prop=info%7Cpageprops&inprop=url&ppprop=disambiguation&redirects=&titles=Guardians+of+the+Galaxy+Vol.+3&format=json&action=query HTTP/1.1" 301 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): en.wikipedia.org:443
DEBUG:urllib3.connectionpool:https://en.wikipedia.org:443 "GET /w/api.php?prop=info%7Cpageprops&inprop=url&ppprop=disambiguation&redirects=&titles=Guardians+of+the+Galaxy+Vol.+3&format=json&action=query HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): en.wikipedia.org:80
DEBUG:urllib3.connectionpool:http://en.wikipedia.org:80 "GET /w/api.php?prop=extracts%7Crevisions&explaintext=&rvprop=ids&titles=Guardians+of+the+Galaxy+Vol.+3&format=json&action=query HTTP/1.1" 301 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): en.wikipedia.org:443
DEBUG:urllib3.connection

In [22]:
from llama_index.llms.openai_like import OpenAILike
from llama_index.core.llms import ChatMessage

In [ ]:
llm_qwen = OpenAILike(
  api_base="https://llms-inference.innkube.fim.uni-passau.de",
  api_key="enter the api key",
  model="qwen2.5")

In [24]:
%%time
messages = [
    ChatMessage(
        role="system", content="You are a assitant that helps to provide knowledge and information."
    ),
    ChatMessage(role="user", content="Tell me more about Rocket"),
]
resp = llm_qwen.chat(messages)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/completions', 'files': None, 'idempotency_key': 'stainless-python-retry-2f7b942e-ca40-432f-93ba-b2e0f741f597', 'json_data': {'model': 'qwen2.5', 'prompt': 'system: You are a assitant that helps to provide knowledge and information.\nuser: Tell me more about Rocket\nassistant: ', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://llms-inference.innkube.fim.uni-passau.de/completions
DEBUG:httpcore.connection:connect_tcp.started host='llms-inference.innkube.fim.uni-passau.de' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x000001D143C43190>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x000001D143A558C0> server_hostname='llms-inference.innkube.fim.uni-passau.de' timeout=60.0
DEBUG:httpcore.connection:start_tls.comple

In [25]:
# Display the response 
print(resp.message.blocks[0].text)

Certainly! "Rocket" can refer to several different things, but most commonly it refers to a vehicle or device designed to be propelled by the ejection of gases produced by combustion. Here are some key points about rockets:

1. **Basic Principle**:
   - Rockets operate on the principle of action and reaction (Newton's Third Law of Motion). The expulsion of gas from the rocket engine produces an equal and opposite force, propelling the rocket forward.

2. **Types of Rockets**:
   - **Solid-Fuel Rockets**: Use a solid propellant that is ignited to produce thrust.
   - **Liquid-Fuel Rockets**: Use liquid propellants, which are fed into the combustion chamber where they mix and burn.
   - **Hybrid Rockets**: Combine elements of both solid and liquid fuel rockets.

3. **Uses**:
   - **Space Exploration**: Rockets are essential for launching satellites, space probes, and crewed missions into space.
   - **Military Applications**: Missiles and other military hardware often use rocket technolo

In [26]:
from llama_index.graph_stores.neo4j import Neo4jGraphStore
from llama_index.core.storage.storage_context import StorageContext

In [28]:
# Initialize the Neo4jGraphStore with your Neo4j database credentials
# Make sure to replace the username, password, and URL with your actual Neo4j database

graph_store = Neo4jGraphStore(
    username="neo4j",
    password="neo4j123",
    url="bolt://localhost:7687",
    database="neo4j"
)

storage_context = StorageContext.from_defaults(graph_store=graph_store)

DEBUG:neo4j.pool:[#0000]  _: <POOL> created, direct address IPv4Address(('localhost', 7687))
DEBUG:neo4j:[#0000]  _: <WORKSPACE> routing towards fixed database: None
DEBUG:neo4j:[#0000]  _: <WORKSPACE> pinning database: None
DEBUG:neo4j.pool:[#0000]  _: <POOL> acquire direct connection, access_mode='READ', database=AcquisitionDatabase(name=None, guessed=False)
DEBUG:neo4j.pool:[#0000]  _: <POOL> trying to hand out new connection
DEBUG:neo4j.io:[#0000]  _: <RESOLVE> in: localhost:7687
DEBUG:neo4j.io:[#0000]  _: <RESOLVE> dns resolver out: [::1]:7687
DEBUG:neo4j.io:[#0000]  C: <OPEN> [::1]:7687
DEBUG:neo4j.io:[#0000]  S: <ERROR> ConnectionRefusedError 10061 'Es konnte keine Verbindung hergestellt werden, da der Zielcomputer die Verbindung verweigerte' None 10061 None
DEBUG:neo4j.io:[#0000]  C: <CLOSE> [::1]:7687
DEBUG:neo4j.io:[#0000]  S: <CONNECTION FAILED> [::1]:7687 ServiceUnavailable: Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [WinError 10061] 

In [30]:
# checking the available methods and attributes of the Neo4jGraphStore and NebulaGraphStore
print(dir(Neo4jGraphStore))
print(dir(NebulaGraphStore))

['__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_is_protocol', '_is_runtime_protocol', 'client', 'close', 'delete', 'get', 'get_rel_map', 'get_schema', 'persist', 'query', 'refresh_schema', 'schema', 'upsert_triplet']
['__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__parameters__', '__reduce__', '__reduc

In [ ]:
# Set global settings (optional, if not passed locally)
# We are using the embedding model and LLM defined above embed_model_snow
Settings.llm = llm_qwen
Settings.embed_model = embed_model_snow
Settings.chunk_size = 512

In [32]:
%%time
# NOTE: can take a while!
# Create a KnowledgeGraphIndex from the documents
# This will create a knowledge graph index from the documents using the specified storage context
index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=2,
)

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Guardians of the Galaxy Vol. 3 (marketed as Gua...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: His return was publicly revealed in March 2019,...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: The Guardians travel to Orgocorp's headquarters...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: They are followed by Ayesha and Adam after the ...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: The other Guardians orchestrate a rescue, which...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: The new Guardians—Rocket, Groot, Kraglin, Cosmo...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Dave Bautista as Drax the Destroyer:A member of...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Pom Klementieff as Mantis: A member of the Guar...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Poulter believed there was "a lot 

In [33]:
# Persist the index to the storage context
# This will save the index to the storage context, allowing it to be reused later
index.storage_context.persist()

DEBUG:fsspec.local:open file: c:/Research/PHD_Related_research/Experiments/KG_with_Llamaindex/storage/docstore.json
DEBUG:fsspec.local:open file: c:/Research/PHD_Related_research/Experiments/KG_with_Llamaindex/storage/index_store.json
DEBUG:fsspec.local:open file: c:/Research/PHD_Related_research/Experiments/KG_with_Llamaindex/storage/default__vector_store.json
DEBUG:fsspec.local:open file: c:/Research/PHD_Related_research/Experiments/KG_with_Llamaindex/storage/image__vector_store.json


In [34]:
# Create a query engine from the index
# This will allow you to query the knowledge graph index
query_engine = index.as_query_engine(
    include_text=False, response_mode="tree_summarize"
)

# Query the knowledge graph index
# This will return a response based on the query and the knowledge graph index
response = query_engine.query("Tell me more about Rocket")


DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/completions', 'files': None, 'idempotency_key': 'stainless-python-retry-3bf14a44-ace7-4afb-98e6-f09d85696e35', 'json_data': {'model': 'qwen2.5', 'prompt': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nTell me more about Rocket\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n", 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://llms-inference.innkube.fim.uni-passau.de/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='llms-inference.innkube.fim.uni-passau.de' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:con

In [35]:
display(Markdown(f"<b>{response}</b>"))

<b>Based on the provided context, there is no direct information about Rocket. However, given that the context revolves around the High Evolutionary, who experiments on and creates various lifeforms, it's possible to infer some connections:

- The High Evolutionary is known for experimenting on and creating hybrid creatures.
- He aims to enhance and anthropomorphize animal lifeforms.

Rocket, a character from the Marvel Cinematic Universe (MCU), is a genetically modified raccoon who was experimented on. While not explicitly mentioned in the provided context, it's likely that Rocket could be one of the many lifeforms that the High Evolutionary has experimented on or created, given his background and the High Evolutionary’s goals.

If you have more specific information about Rocket or any other related entities, please let me know!</b>

In [36]:
# Create a query engine from the index
# This will allow you to query the knowledge graph index including text = True making the response more informative 
query_engine = index.as_query_engine(
    include_text=True, response_mode="tree_summarize"
)

response = query_engine.query("Tell me more about Rocket")

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/completions', 'files': None, 'idempotency_key': 'stainless-python-retry-a7bd2195-162b-4391-9d92-870b1b6cf95b', 'json_data': {'model': 'qwen2.5', 'prompt': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nTell me more about Rocket\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n", 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://llms-inference.innkube.fim.uni-passau.de/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='llms-inference.innkube.fim.uni-passau.de' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:con

In [37]:
display(Markdown(f"<b>{response}</b>"))

<b>Rocket is a key character in the Marvel Cinematic Universe (MCU), particularly known for his role in the "Guardians of the Galaxy" films and the broader MCU series. Here are some detailed points about Rocket:

1. **Background**:
   - **Found and Experimented On**: Rocket was found and experimented on by the High Evolutionary, a character described as one of the cruelest villains in the MCU.
   - **Aimed to Enhance and Anthropomorphize**: The High Evolutionary aimed to enhance and anthropomorphize animal lifeforms, which is how Rocket came to be.

2. **Character Traits**:
   - **Hybrid Creature**: Rocket is a genetically modified raccoon with enhanced intelligence, strength, and combat skills.
   - **Personality**: He is known for his sharp wit, sarcasm, and a tough exterior that often masks deep emotional pain and a sense of abandonment due to his past.

3. **Role in the MCU**:
   - **Guardians of the Galaxy**: Rocket is one of the core members of the Guardians of the Galaxy, alongside Star-Lord, Gamora, Drax, and Groot.
   - **Combat Skills**: He is an expert marksman and a skilled fighter, often providing tactical support and humor in battle situations.

4. **Backstory**:
   - **Orgocorp**: The High Evolutionary's organization, Orgocorp, was responsible for Rocket's creation and the experiments that shaped him.
   - **Painful Past**: Rocket's past is marked by the cruel experiments he endured, which have left him with a deep-seated mistrust of authority and a strong sense of independence.

5. **Portrayal**:
   - **Voice Actor**: Rocket is voiced by Bradley Cooper in the films.
   - **Motion Capture**: The physical performance for Rocket is provided by Sean Gunn, who also plays Kraglin in the MCU.

6. **Significant Moments**:
   - **Guardians of the Galaxy (2014)**: Rocket's first appearance where he and Groot are introduced as part of a group of bounty hunters.
   - **Avengers: Infinity War (2018) and Endgame (2019)**: Rocket plays crucial roles in these films, particularly in the battle against Thanos.

7. **Future Appearances**:
   - **Guardians of the Galaxy Vol. 3**: James Gunn, who directed the first two "Guardians" films, is set to return to write and direct this installment, which will likely delve deeper into Rocket's backstory and his relationship with the other Guardians.

Rocket's complex character and compelling backstory make him a fan favorite in the MCU, known for his humor, loyalty, and deep emotional depth.</b>

## Store and visualize the graph in the html format

In [60]:
from neo4j import GraphDatabase

# Connect to the database
uri = "bolt://localhost:7687"
username = "neo4j"
password = "neo4j123"

driver = GraphDatabase.driver(uri, auth=(username, password))

DEBUG:neo4j.pool:[#0000]  _: <POOL> created, direct address IPv4Address(('localhost', 7687))


In [ ]:
# Function to fetch the graph data
# This function retrieves nodes and relationships from the Neo4j database
def fetch_graph(tx, limit=50):
    query = """
    MATCH (n)-[r]->(m)
    RETURN n, r, m
    LIMIT $limit
    """
    result = tx.run(query, limit=limit)
    data = []

    for record in result:
        data.append({
            "source": dict(record["n"]),
            "relation": record["r"].type,
            "target": dict(record["m"])
        })

    return data

# Safely extract and store the graph data
with driver.session() as session:
    graph_data = session.read_transaction(fetch_graph)

# Now build the sets for visualization
nodes = set()
edges = []

for entry in graph_data:
    source = entry["source"].get("name", str(entry["source"]))
    target = entry["target"].get("name", str(entry["target"]))
    relation = entry["relation"]

    nodes.add(source)
    nodes.add(target)
    edges.append((source, target, relation))

DEBUG:neo4j:[#0000]  _: <WORKSPACE> routing towards fixed database: None
DEBUG:neo4j:[#0000]  _: <WORKSPACE> pinning database: None
DEBUG:neo4j.pool:[#0000]  _: <POOL> acquire direct connection, access_mode='READ', database=AcquisitionDatabase(name=None, guessed=False)
DEBUG:neo4j.pool:[#F88E]  _: <POOL> picked existing connection bolt-25
DEBUG:neo4j.pool:[#F88E]  _: <POOL> checked re_auth auth=None updated=False force=False
DEBUG:neo4j.pool:[#F88E]  _: <POOL> handing out existing connection
DEBUG:neo4j.io:[#F88E]  C: BEGIN {'mode': 'r'}
DEBUG:neo4j.io:[#F88E]  _: <CONNECTION> client state: READY > TX_READY_OR_TX_STREAMING
DEBUG:neo4j.io:[#F88E]  S: SUCCESS {}
DEBUG:neo4j.io:[#F88E]  _: <CONNECTION> server state: READY > TX_READY_OR_TX_STREAMING
DEBUG:neo4j.io:[#F88E]  C: RUN '\n    MATCH (n)-[r]->(m)\n    RETURN n, r, m\n    LIMIT $limit\n    ' {'limit': 50} {}
DEBUG:neo4j.io:[#F88E]  C: PULL {'n': 1000}
DEBUG:neo4j.io:[#F88E]  S: SUCCESS {'t_first': 0, 'fields': ['n', 'r', 'm'], 'qid

C:\Users\Administrator\AppData\Local\Temp\ipykernel_17196\2671942141.py:21: DeprecationWarning: read_transaction has been renamed to execute_read
  graph_data = session.read_transaction(fetch_graph)


In [ ]:
from pyvis.network import Network
from IPython.display import IFrame

# Create a PyVis network graph
# This will visualize the graph data using PyVis

net = Network(notebook=True, height="600px", width="100%", cdn_resources='in_line')

for node in nodes:
    net.add_node(node, label=node)

for source, target, rel in edges:
    net.add_edge(source, target, label=rel)

# Save the graph to an HTML file
# This will create an interactive HTML file that can be viewed in a web browser
net.show("neo4j_graph.html")



neo4j_graph.html
